In [1]:
from Tirto import Tirto #class news Scrapper
from datetime import timedelta, date, datetime
from IPython.display import clear_output

In [2]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [3]:
obj = Tirto()
end_date = date(2018, 8, 8)
start_date = date(2018, 2, 12)
init_page = 2

In [5]:
for single_date in daterange(start_date, end_date):
    clear_output()
    print(single_date.strftime("%Y-%m-%d"))
    obj.getAllBerita([], init_page, single_date.strftime("%Y-%m-%d"))
    init_page = 1

2018-02-12
page  2
https://tirto.id/indeks/2?date=2018-02-12
https://tirto.id/pilgub-sumut-jr-saragih-tak-lolos-tersisa-duel-djarot-vs-edy-cEGv
Insert berita  Pilgub Sumut: JR Saragih Tak Lolos, Tersisa Duel Djarot vs Edy
salah2
https://tirto.id/hasil-piala-presiden-2018-persija-jakarta-lolos-ke-final-cEGw
Insert berita  Hasil Piala Presiden 2018: Persija Jakarta Lolos ke Final
salah2
https://tirto.id/usai-demo-sopir-angkot-m-44-dan-dishub-capai-hasil-kesepakatan-cEGm
Insert berita  Usai Demo, Sopir Angkot M-44 dan Dishub Capai Hasil Kesepakatan
salah2
https://tirto.id/kpu-ntt-marianus-sae-tetap-jadi-cagub-ntt-periode-2018-2023-cEGH
Insert berita  KPU NTT: Marianus Sae Tetap jadi Cagub NTT Periode 2018-2023
salah2
https://tirto.id/nokia-8-resmi-masuk-indonesia-13-februari-cEGf
Insert berita  Nokia 8 Resmi Masuk Indonesia 13 Februari
salah2
https://tirto.id/demo-angkot-m-44-transjakarta-bikin-pendapatan-sopir-berkurang-cEGb
Insert berita  Demo Angkot M-44: Transjakarta Bikin Pendapatan 

ValueError: time data '' does not match format '%d %B %Y'